# <center>Loading Github Data into a relational DataBase<center>
## <center>Bentaleb Saad - YouCode<center>

In [188]:
import pandas as pd
import pyodbc

In [189]:
# loading data
commits = pd.read_csv('data/commits.csv')
repos = pd.read_csv('data/repos.csv')
languages = pd.read_csv('data/languages.csv')

## Connecting to the SQL server using pyodbc
---

In [190]:
driver = 'SQL Server'
server = 'LAPTOP-K8C2EPLP\SQLEXPRESS'
database = 'github_db'


connection_string = f"""
                            DRIVER={driver};
                            SERVER={server};
                            DATABASE={database};
                            Trusted_Connection=yes;
"""
connection = pyodbc.connect(connection_string)
cursor = connection.cursor()
cursor



## Creating the tables using the .sql file of creation transaction
---

In [191]:
# check if the tables were already created
tables = cursor.tables(tableType='TABLE', schema='dbo').fetchall()
tables_names = [table.table_name for table in tables if table.table_schem == 'dbo']
tables_names

[]

In [192]:
# execute the .sql file
if not tables_names :
    cursor.execute(open('github_data_base_creation.sql').read()[3:]).commit()

In [193]:
# check if the tables were created
tables = cursor.tables(tableType='TABLE', schema='dbo').fetchall()
tables_names = [table.table_name for table in tables if table.table_schem == 'dbo']
tables_names

['Commits',
 'Contributions',
 'Contributors',
 'Language_usage',
 'Languages',
 'Owner',
 'Repositories']

In [194]:
# check columns for each table
for table_name in tables_names:
    columns = cursor.columns(table=table_name, schema='dbo').fetchall()
    print(f'Columns for {table_name}:')
    for column in columns:
        print(column.column_name)
    print('\n')

Columns for Commits:
repos_url
id
author_name
author_date
committer_name
committer_date


Columns for Contributions:
repo_url
contributor_id
count


Columns for Contributors:
id
name
profile_url


Columns for Language_usage:
repo_url
language_id
bytes_of_code


Columns for Languages:
id
name


Columns for Owner:
id
name
profile_url


Columns for Repositories:
name
url
description
stars
created_at
language
forks
watchers
open_issues
owner_id
contributors_count




# Inserting data from the dataframes to the database

In [195]:
# a function to insert data into a table
def insert_data(table_name, data):
    columns = cursor.columns(table=table_name, schema='dbo').fetchall()
    columns_names = [column.column_name for column in columns]
    columns_names = ', '.join(columns_names)
    values = ', '.join(['?' for _ in range(len(columns_names.split(', ')))])
    query = f"INSERT INTO {table_name} ({columns_names}) VALUES ({values})"
    cursor.executemany(query, data)
    connection.commit()


### filling the tables with data depending on the order of dependencie:
### Languages -> Contributors -> Owner -> Repositories -> Commits -> Language_usage -> Contributions
---

## <center>1 - Languages table<center>
---

In [196]:
# get the list of unique languages from the languages columns names
languages_names = languages.columns[1:].tolist()
# generate id for each language
languages_ids = [i for i in range(1, len(languages_names) + 1)]
# create a list of tuples with the language id and the language name
languages_names = list(zip(languages_ids, languages_names))
# insert data into the Languages table
insert_data('Languages', languages_names)
# check if the data was inserted
cursor.execute('SELECT * FROM Languages').fetchmany(5)

[(1, 'Filebench WML'),
 (2, 'Nix'),
 (3, 'Awk'),
 (4, 'Common Lisp'),
 (5, 'Classic ASP')]

## <center>2 - Contributors table<center>
---

In [197]:
# generate id for each contributor
commits["contributor_id"] = commits["author_name"].astype('category').cat.codes
commits["name"] = commits["author_name"]
commits["profile_url"] = ("https://github.com/" + commits["author_name"]).replace(" ", "+").astype(str)
contributors = commits[['contributor_id', 'name', 'profile_url']].drop_duplicates()
# as tuples
contributors = [tuple(contributor) for contributor in contributors.values.tolist()]
# insert data into the Contributors table
insert_data('Contributors', contributors)
cursor.execute('SELECT * FROM Contributors').fetchmany(5)

[(0, '#WANG JIANYI#', 'https://github.com/#WANG JIANYI#'),
 (1, '(Bill) Yuchen Lin', 'https://github.com/(Bill) Yuchen Lin'),
 (2, '(Holloway) Chew, Kean Ho', 'https://github.com/(Holloway) Chew, Kean Ho'),
 (3, '(cdep)illabout', 'https://github.com/(cdep)illabout'),
 (4, '(yunjin)', 'https://github.com/(yunjin)')]

## <center>3 - Owner table<center>
---

In [198]:
# Generate id for each owner
repos["owner_id"] = repos["owner"].astype('category').cat.codes
# Get the list of unique owners from the owner column
owners = repos[['owner_id', 'owner']].drop_duplicates(subset=['owner'])
owners['id'] = owners['owner_id']
owners['name'] = owners['owner']
owners['profile_url'] = ("https://github.com/" + owners["owner"]).astype(str)
owners = owners[['id', 'name', 'profile_url']]
# Convert owners DataFrame to list of tuples
owners_list = [tuple(row) for row in owners.values.tolist()]
# Insert data into the Owners table
insert_data('Owner', owners_list)
cursor.execute('SELECT * FROM Owner').fetchmany(5)

[(0, '000miix', 'https://github.com/000miix'),
 (1, '0015', 'https://github.com/0015'),
 (2, '007revad', 'https://github.com/007revad'),
 (3, '02JanDal', 'https://github.com/02JanDal'),
 (4, '050310Y', 'https://github.com/050310Y')]

## <center>4 - Repositories table<center>
---

In [199]:
repos = repos[['name', 'url', 'description', 'stars', 'created_at', 'language', 'forks', 'watchers', 'open_issues', 'owner_id', 'contributors_count']]
# as tuples
repos = [tuple(repo) for repo in repos.values.tolist()]
# insert data into the Repositories table
insert_data('Repositories', repos)
cursor.execute('SELECT * FROM Repositories').fetchmany(5)

[('DailyGrapher', 'https://github.com/000miix/DailyGrapher', 'A Hacs Custom Card that shows a calendar on a clock', 31, '2023-05-05', 'JavaScript', 1, 31, 2, 0, 2),
 ('ChatGPT_Client_For_Arduino', 'https://github.com/0015/ChatGPT_Client_For_Arduino', 'Library for communication with ChatGPT', 46, '2023-03-07', 'C++', 1, 46, 0, 1, 1),
 ('Synology_enable_M2_volume', 'https://github.com/007revad/Synology_enable_M2_volume', 'Enable creating volumes with non-Synology M.2 drives', 115, '2023-03-28', 'Shell', 12, 115, 7, 2, 2),
 ('Synology_HDD_db', 'https://github.com/007revad/Synology_HDD_db', "Add your HDD, SSD and NVMe drives to your Synology's compatible drive database", 652, '2023-02-26', 'Shell', 39, 652, 3, 2, 3),
 ('Synology_M2_volume', 'https://github.com/007revad/Synology_M2_volume', 'Easily create an M.2 volume on Synology NAS', 261, '2023-03-22', 'Shell', 19, 261, 2, 2, 2)]

## <center>5 - Commits table<center>
---

In [206]:
# load commits data
commits = pd.read_csv('data/commits.csv')
# get the list of unique languages from the languages columns names
commits = commits[['url', 'id', 'author_name', 'author_date', 'committer_name', 'committer_date']]
# drop duplicates by id
commits = commits.drop_duplicates(subset=['id'])
# generate id for each commit
commits["id"] = commits["id"].astype('category').cat.codes
# as tuples
commits = [tuple(commit) for commit in commits.values.tolist()]
# insert data into the Commits table
insert_data('Commits', commits)
cursor.execute('SELECT * FROM Commits').fetchmany(5)

[('https://github.com/Totoro97/f2-nerf', 0, 'totoro97', '2023-04-24', 'totoro97', '2023-04-24'),
 ('https://github.com/ki-lang/ki', 1, 'ctxz', '2023-06-06', 'ctxz', '2023-06-06'),
 ('https://github.com/FuncJin/Covid19-Visualization', 2, 'Ssylqwj1212', '2023-04-05', 'Ssylqwj1212', '2023-04-05'),
 ('https://github.com/e4s2022/e4s', 3, 'e4s2022', '2023-03-29', 'GitHub', '2023-03-29'),
 ('https://github.com/ashyanSpada/expression_engine_rs', 4, 'ausarwang', '2023-03-19', 'ausarwang', '2023-03-19')]

## <center>6 - Language_usage table (relation entity)<center>
---

In [226]:
# get the list of unique languages from the languages columns names
languages_names = languages.columns[1:].tolist()
# generate id for each language
languages_ids = [i for i in range(1, len(languages_names) + 1)]
# create a list of tuples with the language id and the language name
languages_names = zip(languages_ids, languages_names)
languages_names = list(languages_names)
#as data frame
languages_names = pd.DataFrame(languages_names, columns=['id', 'name'])

In [218]:
# create a dataframe with the url and a row for each language with the number of bytes
languages1 = languages.melt(id_vars=['url'], var_name='language', value_name='bytes')
# drop rows with 0 bytes
languages1 = languages1[languages1['bytes'] != 0]
# order by url
languages1 = languages1.sort_values(by=['url'])
# get languages ids based on the language name for each language
languages1 = languages1.merge(languages_names, left_on='language', right_on='name')
languages1 = languages1[['url', 'id', 'bytes']]
# as tuples
languages1 = [tuple(language) for language in languages1.values.tolist()]
# insert data into the Language_usage table
insert_data('Language_usage', languages1)
cursor.execute('SELECT * FROM Language_usage').fetchmany(5)

[('https://github.com/000miix/DailyGrapher', 117, 15018),
 ('https://github.com/0hq/ephemeral-notes', 117, 9972),
 ('https://github.com/0prrr/Malwear-Sweet', 117, 103),
 ('https://github.com/0ut0flin3/Reptyl', 117, 7704),
 ('https://github.com/0wQ/air780e-forwarder', 117, 3380334)]

## <center>7 - Contributions table (relation entity)<center>
---

In [220]:
# load repos data
repos = pd.read_csv('data/repos.csv')
# create a dataframe with the repo_url and contributor_id and count from repos[['contributors', 'contributions_count']] ex ["'gnwx'", " 'sumn2u'", " 'Anuj-Khadka'"] and [10, 1, 1]
contributions = repos[['url', 'contributors']]
# type of the contributors column
contributions['contributors'].astype(str)
# split the contributors column
contributions['contributors'] = contributions['contributors'].astype(str).str.split(',')
# explode the contributors column
contributions = contributions.explode('contributors')
# remove the spaces
contributions['contributors'] = contributions['contributors'].str.strip()
# remove the quotes
contributions['contributors'] = contributions['contributors'].str.replace("'", "")

contributions1 = repos[['url', 'contributions_count']]
# split the contributions_count column
contributions1['contributions_count'] = contributions1['contributions_count'].astype(str).str.split(',')
# explode the contributions_count column
contributions1 = contributions1.explode('contributions_count')
# remove the spaces
contributions1['contributions_count'] = contributions1['contributions_count'].str.strip()
# explode the contributions_count column 
contributions1 = contributions1.explode('contributions_count')

contributions["contributions_count"]  = contributions1["contributions_count"]

# remove spaces and quotes and [ and ] from the contributors column
contributions['contributors'] = contributions['contributors'].str.replace("'", "")
contributions['contributors'] = contributions['contributors'].str.replace("\"", "")
contributions['contributors'] = contributions['contributors'].str.replace("[", "")
contributions['contributors'] = contributions['contributors'].str.replace("]", "")
contributions['contributors'] = contributions['contributors'].str.strip()

contributions['contributions_count'] = contributions['contributions_count'].str.replace("[", "")
contributions['contributions_count'] = contributions['contributions_count'].str.replace("]", "")
contributions['contributions_count'] = contributions['contributions_count'].str.strip()

# get the contributor_id from the Contributors table using the find by name request if it exists
contributions['contributor_id'] = contributions['contributors'].apply(lambda x: cursor.execute(f"SELECT id FROM Contributors WHERE name = '{x}'").fetchone())
# drop rows with no contributor_id
contributions = contributions.dropna(subset=['contributor_id'])

C:\Users\YouCode\AppData\Local\Temp\ipykernel_4844\1663301996.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contributions['contributors'] = contributions['contributors'].astype(str).str.split(',')
C:\Users\YouCode\AppData\Local\Temp\ipykernel_4844\1663301996.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contributions1['contributions_count'] = contributions1['contributions_count'].astype(str).str.split(',')


In [224]:
# convert the contributor_id to int after removing the brackets
contributions['contributor_id'] = contributions['contributor_id'].astype(str).str.replace("(\'", "")
contributions['contributor_id'] = contributions['contributor_id'].str.replace("\',)", "")
contributions['contributor_id'] = contributions['contributor_id'].str.replace(",)", "")
contributions['contributor_id'] = contributions['contributor_id'].str.replace("(", "")
contributions['contributor_id'] = contributions['contributor_id'].str.strip()
contributions['contributor_id'] = contributions['contributor_id'].astype(int)
# drop the contributors column
contributions2 = contributions[['url', 'contributor_id', 'contributions_count']]
# as tuples
contributions2 = [tuple(contribution) for contribution in contributions2.values.tolist()]
# insert data into the Contributions table
insert_data('Contributions', contributions2)

In [225]:
cursor.execute('SELECT * FROM Contributions').fetchmany(5)

[('https://github.com/sumn2u/learn-javascript', 23390, 83),
 ('https://github.com/rupali-codes/LinksHub', 26820, 6),
 ('https://github.com/LondheShubham153/90DaysOfDevOps', 11032, 167),
 ('https://github.com/LondheShubham153/90DaysOfDevOps', 18553, 3),
 ('https://github.com/LondheShubham153/90DaysOfDevOps', 27657, 1)]

## Trying some requests to get data from the database
---

In [228]:
# get the top 10 repositories with the most stars along with their owner names
query = """
SELECT TOP 10
    r.name AS RepositoryName,
    r.stars AS Stars,
    r.created_at AS CreatedAt,
    o.name AS OwnerName
FROM Repositories r
INNER JOIN Owner o ON r.owner_id = o.id
ORDER BY r.stars DESC;
"""
cursor.execute(query).fetchall()

[('Auto-GPT', 142390, '2023-03-16', 'Significant-Gravitas'),
 ('the-algorithm', 58374, '2023-03-27', 'twitter'),
 ('node', 51652, '2023-02-01', 'base-org'),
 ('gpt4all', 48173, '2023-03-27', 'nomic-ai'),
 ('gpt4free', 41894, '2023-03-29', 'xtekky'),
 ('segment-anything', 35542, '2023-03-23', 'facebookresearch'),
 ('gpt-engineer', 35368, '2023-04-29', 'AntonOsika'),
 ('ChatGPT-Next-Web', 34814, '2023-03-10', 'Yidadaa'),
 ('TaskMatrix', 33500, '2023-03-02', 'microsoft'),
 ('llama.cpp', 33293, '2023-03-10', 'ggerganov')]

In [229]:
# get the top 5 contributors based on the total count of their contributions
query = """
SELECT TOP 5
    c.name AS ContributorName,
    SUM(co.count) AS TotalContributions
FROM Contributors c
INNER JOIN Contributions co ON c.id = co.contributor_id
GROUP BY c.name
ORDER BY TotalContributions DESC;
"""
cursor.execute(query).fetchall()

[('MoonLight0127', 12700),
 ('vfarid', 6679),
 ('jxxghp', 6286),
 ('drshahizan', 5214),
 ('adamlui', 4237)]

In [231]:
# get the number of repositories for each language along with the total count of stars for each language
query = """
SELECT
    l.name AS LanguageName,
    COUNT(r.url) AS RepoCount,
    SUM(r.stars) AS TotalStars
FROM Languages l
LEFT JOIN Language_usage lu ON l.id = lu.language_id
LEFT JOIN Repositories r ON lu.repo_url = r.url
GROUP BY l.name ORDER BY TotalStars DESC;
"""
cursor.execute(query).fetchmany(5)

[('Python', 6250, 1840208),
 ('JavaScript', 4381, 1408669),
 ('Shell', 3853, 1221808),
 ('CSS', 2974, 1057348),
 ('Dockerfile', 1739, 934787)]

In [232]:
# get the latest 5 commits for a given repository

query = """
DECLARE @RepositoryURL VARCHAR(555) = 'https://github.com/sumn2u/learn-javascript'

SELECT TOP 5
    c.id AS CommitID,
    c.author_name AS AuthorName,
    c.author_date AS AuthorDate,
    c.committer_name AS CommitterName,
    c.committer_date AS CommitterDate
FROM Commits c
WHERE c.repos_url = @RepositoryURL
ORDER BY c.author_date DESC;
"""
cursor.execute(query).fetchall()

[(284676, 'gnwx', '2023-07-05', 'Suman Kunwar', '2023-07-05'),
 (210364, 'gnwx', '2023-07-04', 'Suman Kunwar', '2023-07-04'),
 (245001, 'gnwx', '2023-07-03', 'Suman Kunwar', '2023-07-03'),
 (316900, 'Suman Kunwar', '2023-07-03', 'GitHub', '2023-07-03'),
 (145286, 'gnwx', '2023-06-27', 'Suman Kunwar', '2023-06-27')]